In [1]:
# -*- coding: utf-8 -*-
"""
@author: gr
"""
import os
import numpy as np
import tensorflow as tf
import input_train_val_split
import model


N_CLASSES = 2
IMG_W = 208  # resize the image, if the input image is too large, training will be very slow.
IMG_H = 208
RATIO = 0.3 # take 20% of dataset as validation data 
BATCH_SIZE = 20
CAPACITY = 1000
MAX_STEP = 50000 # with current parameters, it is suggested to use MAX_STEP>10k
learning_rate = 0.0001 # with current parameters, it is suggested to use learning rate<0.0001


def run_training():
    
    train_dir = '/home/yanqinhong/DL/eye/data/train/'
    logs_train_dir = '/home/yanqinhong/DL/eye/logs/train/'
    logs_val_dir = '/home/yanqinhong/DL/eye/logs/val/'
    
    train, train_label, val, val_label = input_train_val_split.get_files(train_dir, RATIO)
    train_batch, train_label_batch = input_train_val_split.get_batch(train,
                                                  train_label,
                                                  IMG_W,
                                                  IMG_H,
                                                  BATCH_SIZE, 
                                                  CAPACITY)
    val_batch, val_label_batch = input_train_val_split.get_batch(val,
                                                  val_label,
                                                  IMG_W,
                                                  IMG_H,
                                                  BATCH_SIZE, 
                                                  CAPACITY)
    
    x = tf.placeholder(tf.float32, shape=[BATCH_SIZE, IMG_W, IMG_H, 3])
    y_ = tf.placeholder(tf.int32, shape=[BATCH_SIZE])
    
    logits = model.inference(x, BATCH_SIZE, N_CLASSES)
    loss = model.losses(logits, y_)        
    train_op = model.training(loss, learning_rate)
    acc = model.evaluation(logits, y_)
        
    with tf.Session() as sess:
        saver = tf.train.Saver()
        sess.run(tf.global_variables_initializer())
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(sess=sess, coord=coord)
        
        summary_op = tf.summary.merge_all()        
        train_writer = tf.summary.FileWriter(logs_train_dir, sess.graph)
        val_writer = tf.summary.FileWriter(logs_val_dir, sess.graph)
    
        try:
            
            for step in np.arange(MAX_STEP):
                if coord.should_stop():
                    break
                
                tra_images,tra_labels = sess.run([train_batch, train_label_batch])
                _, tra_loss, tra_acc = sess.run([train_op, loss, acc],
                                                feed_dict={x:tra_images, y_:tra_labels})
                if step % 50 == 0:
                    print('Step %d, train loss = %.2f, train accuracy = %.2f%%' %(step, tra_loss, tra_acc*100.0))
#                     summary_str = sess.run(summary_op)
#                     train_writer.add_summary(summary_str, step)
                    
                if step % 200 == 0 or (step + 1) == MAX_STEP:
                    val_images, val_labels = sess.run([val_batch, val_label_batch])
                    val_loss, val_acc = sess.run([loss, acc], 
                                                 feed_dict={x:val_images, y_:val_labels})
                    print('**  Step %d, val loss = %.2f, val accuracy = %.2f%%  **' %(step, val_loss, val_acc*100.0))
#                     summary_str = sess.run(summary_op)
#                     val_writer.add_summary(summary_str, step)  
                                    
                if step % 2000 == 0 or (step + 1) == MAX_STEP:
                    checkpoint_path = os.path.join(logs_train_dir, 'model.ckpt')
                    saver.save(sess, checkpoint_path, global_step=step)
                    
        except tf.errors.OutOfRangeError:
            print('Done training -- epoch limit reached')
        finally:
            coord.request_stop()           
        coord.join(threads)


from PIL import Image
import matplotlib.pyplot as plt

def get_one_image(train):
    '''Randomly pick one image from training data
    Return: ndarray
    '''
    n = len(train)
    ind = np.random.randint(0, n)
    img_dir = train[ind]

    image = Image.open(img_dir)
    plt.imshow(image)
    image = image.resize([208, 208])
    image = np.array(image)
    return image


def evaluate_one_image():
    '''Test one image against the saved models and parameters
    '''
    
    test_dir = '/home/gr/DL/eye/data/test'
    train, train_label, _, _ = input_train_val_split.get_files(test_dir,0.)
    image_array = get_one_image(train)
    
    with tf.Graph().as_default():
        BATCH_SIZE = 1
        N_CLASSES = 2
        
        image = tf.cast(image_array, tf.float32)
        image = tf.image.per_image_standardization(image)
        image = tf.reshape(image, [1, 208, 208, 3])
        
        logit = model.inference(image, BATCH_SIZE, N_CLASSES)
        
        logit = tf.nn.softmax(logit)
        
        x = tf.placeholder(tf.float32, shape=[208, 208, 3])
        
        logs_train_dir = '/home/gr/DL/eye/logs/train/' 
                       
        saver = tf.train.Saver()
        
        with tf.Session() as sess:
            
            print("Reading checkpoints...")
            ckpt = tf.train.get_checkpoint_state(logs_train_dir)
            if ckpt and ckpt.model_checkpoint_path:
                global_step = ckpt.model_checkpoint_path.split('/')[-1].split('-')[-1]
                saver.restore(sess, ckpt.model_checkpoint_path)
                print('Loading success, global_step is %s' % global_step)
            else:
                print('No checkpoint file found')
            
            prediction = sess.run(logit, feed_dict={x: image_array})
            max_index = np.argmax(prediction)
            if max_index==0:
                print('This is health with possibility %.6f' %prediction[:, 0])
            else:
                print('This is sick with possibility %.6f' %prediction[:, 1])

In [2]:
run_training()

There are 7983 health
There are 1992 sick
Step 0, train loss = 0.69, train accuracy = 14.99%
**  Step 0, val loss = 0.67, val accuracy = 89.94%  **
Step 50, train loss = 0.43, train accuracy = 84.96%
Step 100, train loss = 0.47, train accuracy = 84.96%
Step 150, train loss = 0.55, train accuracy = 75.00%
Step 200, train loss = 0.59, train accuracy = 69.97%
**  Step 200, val loss = 0.49, val accuracy = 79.98%  **
Step 250, train loss = 0.24, train accuracy = 94.92%
Step 300, train loss = 0.50, train accuracy = 79.98%
Step 350, train loss = 0.56, train accuracy = 75.00%
Step 400, train loss = 0.57, train accuracy = 75.00%
**  Step 400, val loss = 0.35, val accuracy = 84.96%  **
Step 450, train loss = 0.43, train accuracy = 84.96%
Step 500, train loss = 0.49, train accuracy = 75.00%
Step 550, train loss = 0.43, train accuracy = 84.96%
Step 600, train loss = 0.54, train accuracy = 75.00%
**  Step 600, val loss = 0.52, val accuracy = 75.00%  **
Step 650, train loss = 0.65, train accuracy = 

In [3]:
import os
import numpy as np
from PIL import Image
import tensorflow as tf
import model

image_W = 208
image_H = 208
CAPACITY = 1000
BATCH_SIZE = 1
N_CLASSES = 2

test_dir = '/home/yanqinhong/DL/eye/data/test' # test data file
logs_train_dir = '/home/yanqinhong/DL/eye/logs/train/' # model file

test_image = []
test_label = []
for file1 in os.listdir(test_dir):
    for file2 in os.listdir(test_dir + '/' + file1):
        if file1 == 'health':
            test_label.append(0)
        else:
            test_label.append(1)
        test_image.append(test_dir+'/'+file1+'/'+file2)
    

print('There are %d test image' %len(test_image))

pred = []

with tf.Graph().as_default():

        x = tf.placeholder(tf.float32, shape=[1, 208, 208, 3])
        logit = model.inference(x, BATCH_SIZE, N_CLASSES)
        logit = tf.nn.softmax(logit)
        
        with tf.Session() as sess:
            saver = tf.train.Saver()
            print("Reading checkpoints...")
            ckpt = tf.train.get_checkpoint_state(logs_train_dir)
            if ckpt and ckpt.model_checkpoint_path:
                global_step = ckpt.model_checkpoint_path.split('/')[-1].split('-')[-1]
                saver.restore(sess, ckpt.model_checkpoint_path)
                print('Loading success, global_step is %s' % global_step)
            else:
                print('No checkpoint file found')
            
            for image in test_image:
                image = Image.open(image)
                image = image.resize((image_W, image_H))
                image = np.array(image)
                image = tf.convert_to_tensor(image)
                image = tf.image.per_image_standardization(image)
                image = tf.cast(image, tf.float32)
                image = tf.reshape(image, [1, 208, 208, 3])
                image = image.eval(session = sess)
                prediction = sess.run(logit, feed_dict={x:image})
                max_index = np.argmax(prediction)
                pred.append(max_index)

# testing accuracy
acc = np.sum(test_label==np.array(pred))/float(len(test_label))
print('testing accuracy: %s' %acc)
        
        

There are 200 test image
Reading checkpoints...
INFO:tensorflow:Restoring parameters from /home/yanqinhong/DL/eye/logs/train/model.ckpt-9999
Loading success, global_step is 9999
testing accuracy: 0.745
